In [22]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

examples = [
    {
        "movie": "기생충",
        "answer" : """
        "director": "봉준호"
        "cast" : "송강호, 이선균, 조여정, 최우식, 박소담"
        "budget": "약 130억 원"
        "box_office" :"약 3,100억 원"
        "genre" : "드라마, 스릴러"
        "synopsis" : "기택의 가족은 전원이 백수로 생활고를 겪고 있다. 큰아들 기우는 친구의 소개로 부잣집 과외교사로 들어가며, 두 가족이 엮이게 된다.
        """,
    },
    {
        "movie": "올드보이",
        "answer" : """
        "director": "박찬욱"
        "cast": "최민식, 유지태, 강혜정"
        "budget": "약 35억 원",
        "box_office": "약 175억 원"
        "genre": "스릴러, 미스터리"
        "synopsis": "15년간 이유도 모른 채 감금된 오대수는 갑자기 풀려나게 되고, 자신을 감금한 자를 찾아 복수를 시작한다.
        """
    },

]


examples_prompt = PromptTemplate.from_template(
    "Human : Tell me about {movie} movie. AI : {answer}",
)

prompt = FewShotPromptTemplate(
    example_prompt= examples_prompt,
    examples = examples,
    suffix = "Tell me about {movie} movie",
    input_variables=["movie"]
)

chain = prompt | chat

chain.invoke({
    "movie" : "부산행"
})

AI: 
        "director": "연상호"
        "cast": "공유, 정유미, 마동석, 김수안"
        "budget": "약 20억 원"
        "box_office": "약 9,000억 원"
        "genre": "액션, 스릴러, 좀비"
        "synopsis": "부산행 열차를 타고 가족을 찾아가는 성현은 갑작스럽게 좀비 바이러스가 퍼지면서 생존을 위한 사투를 벌이게 된다."

AIMessageChunk(content='AI: \n        "director": "연상호"\n        "cast": "공유, 정유미, 마동석, 김수안"\n        "budget": "약 20억 원"\n        "box_office": "약 9,000억 원"\n        "genre": "액션, 스릴러, 좀비"\n        "synopsis": "부산행 열차를 타고 가족을 찾아가는 성현은 갑작스럽게 좀비 바이러스가 퍼지면서 생존을 위한 사투를 벌이게 된다."')

In [31]:
example_prompt = ChatPromptTemplate.from_messages(
    [("human", "Tell me about {movie} movie."), ("ai", "{answer}")]
)

fewshot_chat_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are a movie expert. Provide detailed and structured information about movies in the following format:\n",
            
            "영화 이름, 감독, 주요 출연진, 예산, 흥핵 수익, 장르, 시놉시스""",
        ),
        fewshot_chat_prompt,
        ("human", "tell me about {movie} movie."),
    ]
)

chain = final_prompt | chat


chain.invoke({"movie": "The Avengers"})

{
    "영화 이름": "The Avengers",
    "감독": "조스 웨던",
    "주요 출연진": "로버트 다우니 주니어, 크리스 에반스, 크리스 헴스워스, 스칼릿 요한슨, 마크 러팔로",
    "예산": "약 2억 20백만 달러",
    "흥행 수익": "약 15억 40백만 달러",
    "장르": "액션, 모험, SF",
    "시놉시스": "마블 코믹스의 슈퍼히어로들이 모여 세계의 위기에 맞서 싸우는 이야기로, 아이언맨, 캡틴 아메리카, 토르, 헐크, 블랙 위도우 등이 한데 모여 팀을 이루어 악당으로부터 지구를 구하는 스토리를 그린다."
}

AIMessageChunk(content='{\n    "영화 이름": "The Avengers",\n    "감독": "조스 웨던",\n    "주요 출연진": "로버트 다우니 주니어, 크리스 에반스, 크리스 헴스워스, 스칼릿 요한슨, 마크 러팔로",\n    "예산": "약 2억 20백만 달러",\n    "흥행 수익": "약 15억 40백만 달러",\n    "장르": "액션, 모험, SF",\n    "시놉시스": "마블 코믹스의 슈퍼히어로들이 모여 세계의 위기에 맞서 싸우는 이야기로, 아이언맨, 캡틴 아메리카, 토르, 헐크, 블랙 위도우 등이 한데 모여 팀을 이루어 악당으로부터 지구를 구하는 스토리를 그린다."\n}')